In [2]:
from ema_workbench import Scenario
from ema_workbench import Model, MultiprocessingEvaluator, Policy, Scenario

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.analysis import pairs_plotting
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ema_workbench.analysis import prim
from ema_workbench.analysis import dimensional_stacking

from ema_workbench.analysis import feature_scoring
from pandas.api.types import is_numeric_dtype, is_categorical_dtype
from sklearn import preprocessing
from ema_workbench.em_framework.optimization import (ArchiveLogger,
                                                     EpsilonProgress,
                                                     to_problem, epsilon_nondominated)

from ema_workbench import ScalarOutcome

ema_logging.log_to_stderr(ema_logging.INFO)



In [3]:
df_scenarios= pd.read_csv('../reference_scenarios.csv')

df_scenarios = (
    df_scenarios
    .rename(columns={"scenario.1": "scenario"})  # als hij zo heet
    .set_index("scenario")                        # zet de kolom als index
)

In [4]:
# lees de CSV in met de gevonden policies vanuit de MORDM
policies_df = pd.read_csv("../candidate_policies.csv")

# zet elke rij om in een Policy-object
candidate_policies = [
    Policy(f"cand_{i}", **row.to_dict())
    for i, row in policies_df.iterrows()

    ]

In [19]:
if __name__ == "__main__":
    model, _ = get_model_for_problem_formulation(3)
    scenarios=10
    policies=4
    with MultiprocessingEvaluator(model, n_processes=-1) as evaluator:
        results = evaluator.perform_experiments(scenarios=scenarios, policies=candidate_policies )

        experiments, outcomes = results

[MainProcess/INFO] pool started with 7 workers

100%|████████████████████████████████████████| 150/150 [00:07<00:00, 20.21it/s]
[MainProcess/INFO] terminating pool


In [20]:
cols_unc = [u.name for u in model.uncertainties]


In [21]:
df_scenarios=df_scenarios[cols_unc]
df_scenarios

,discount rate 0,discount rate 1,discount rate 2,A.0_ID flood wave shape,A.1_Bmax,A.1_pfail,A.1_Brate,A.2_Bmax,A.2_pfail,A.2_Brate,A.3_Bmax,A.3_pfail,A.3_Brate,A.4_Bmax,A.4_pfail,A.4_Brate,A.5_Bmax,A.5_pfail,A.5_Brate
scenario,,,,,,,,,,,,,,,,,,,
"(best_case, 1257)",1.5,3.5,2.5,3,240.596860,0.800122,10.0,162.388872,0.248191,10.0,97.503836,0.472066,1.5,182.792381,0.990409,1.5,113.367141,0.943914,1.5
"(median_case, 2101)",3.5,1.5,4.5,60,148.203377,0.946282,10.0,330.511958,0.481342,10.0,321.353772,0.104562,1.0,231.761207,0.301204,1.0,51.527901,0.763490,10.0
"(worst_case, 2038)",1.5,1.5,1.5,67,98.267897,0.749968,10.0,230.389994,0.020571,1.0,200.050942,0.210249,1.0,332.334382,0.504365,1.5,76.649077,0.514293,1.0


In [22]:
candidate_scenarios = [
    Scenario(f"scenario_{idx}", **row.to_dict())
    for idx, row in df_scenarios.iterrows()
]


In [23]:
for scen in candidate_scenarios:
    # print eerst de door jou opgegeven naam
    print("naam:", scen.name)
    # en daarna de waarden
    print(scen)
    print()

naam: scenario_('best_case', 1257)
Scenario({'discount rate 0': 1.5, 'discount rate 1': 3.5, 'discount rate 2': 2.5, 'A.0_ID flood wave shape': 3.0, 'A.1_Bmax': 240.59685979062849, 'A.1_pfail': 0.8001218927421738, 'A.1_Brate': 10.0, 'A.2_Bmax': 162.38887188791776, 'A.2_pfail': 0.2481911050156166, 'A.2_Brate': 10.0, 'A.3_Bmax': 97.50383584323322, 'A.3_pfail': 0.4720661557138879, 'A.3_Brate': 1.5, 'A.4_Bmax': 182.79238065321385, 'A.4_pfail': 0.9904093412209528, 'A.4_Brate': 1.5, 'A.5_Bmax': 113.36714090569951, 'A.5_pfail': 0.9439138828869614, 'A.5_Brate': 1.5})

naam: scenario_('median_case', 2101)
Scenario({'discount rate 0': 3.5, 'discount rate 1': 1.5, 'discount rate 2': 4.5, 'A.0_ID flood wave shape': 60.0, 'A.1_Bmax': 148.20337687059146, 'A.1_pfail': 0.946281506727712, 'A.1_Brate': 10.0, 'A.2_Bmax': 330.5119580334476, 'A.2_pfail': 0.4813418702616602, 'A.2_Brate': 10.0, 'A.3_Bmax': 321.3537718997584, 'A.3_pfail': 0.1045617305588998, 'A.3_Brate': 1.0, 'A.4_Bmax': 231.76120691375925, '

In [26]:
ema_logging.log_to_stderr(ema_logging.INFO)

def optimize(scenario, nfe, model, epsilons):
    results = []
    convergences = []
    problem = to_problem(model, searchover="levers")

    with MultiprocessingEvaluator(model) as evaluator:
        for i in range(5):
            convergence_metrics = [
                ArchiveLogger(
                    "./archives",
                    [l.name for l in model.levers],
                    [o.name for o in model.outcomes],
                    base_filename=f"multi_MORDM_{scenario.name}_seed_{i}.tar.gz",
                ),
                EpsilonProgress(),
            ]

            result, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
                                         convergence=convergence_metrics,
                                         epsilons=epsilons,
                                         reference=scenario)

            results.append(result)
            convergences.append(convergence)

    # merge the results using a non-dominated sort
    reference_set = epsilon_nondominated(results, epsilons, problem)

    return reference_set, convergences



In [29]:
results = []
for scenario in candidate_scenarios:
    epsilons = [0.05,]*len(model.outcomes)

    # note that 100000 nfe is again rather low to ensure proper convergence
    results.append(optimize(scenario, 100, model, epsilons))

[MainProcess/INFO] pool started with 8 workers


  0%|                                                  | 0/100 [00:00<?, ?it/s]INFO | EpsNSGAII starting
INFO | EpsNSGAII finished; Total NFE: 100, Elapsed Time: 0:00:04.361901


100%|████████████████████████████████████████| 100/100 [00:04<00:00, 22.92it/s]
[MainProcess/INFO] optimization completed, found 8 solutions


  0%|                                                  | 0/100 [00:00<?, ?it/s]INFO | EpsNSGAII starting
INFO | EpsNSGAII finished; Total NFE: 100, Elapsed Time: 0:00:04.548336


100%|████████████████████████████████████████| 100/100 [00:04<00:00, 21.98it/s]
[MainProcess/INFO] optimization completed, found 13 solutions


  0%|                                                  | 0/100 [00:00<?, ?it/s]INFO | EpsNSGAII starting
INFO | EpsNSGAII finished; Total NFE: 100, Elapsed Time: 0:00:04.492191


100%|████████████████████████████████████████| 100/100 [00:04<00:00, 22.25it/s]
[MainProcess/INFO] optimization completed, foun

EMAError: Parameter names ['0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0', '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0', '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2', 'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2', 'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2', 'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2', 'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2', 'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2'] not found in archive

In [34]:
from ema_workbench.em_framework.optimization import ArchiveLogger
# if you used base_filename="MORDM.tar.gz" in your logger above, adjust the path:
archive_file = "./archives/multi_MORDM_scenario_('best_case', 1257)_seed_2.tar.gz"

# 1) load everything that’s in that archive
archives = ArchiveLogger.load_archives(archive_file)

# archives is a dict mapping your base_filenames (e.g. "MORDM.tar.gz")
# to ArchiveLogger objects, so pick one:
arch = archives["multi_MORDM_scenario_('best_case', 1257)_seed_2.tar.gz"]

# 2) inspect exactly what your decision variables and outcomes are called
print("   decision variables:\n", arch.experiments.columns.tolist())
print("         outcomes keys:\n", list(arch.outcomes.keys()))

KeyError: "multi_MORDM_scenario_('best_case', 1257)_seed_2.tar.gz"

In [35]:
# 1) load everything in that folder
archives = ArchiveLogger.load_archives("../archives")

# 2) inspect the dict keys so you know how to index it
print("available archives:", list(archives.keys()))

# 3) grab one of them
#    either by matching the filename you see above, or simply:
arch = list(archives.values())[0]

# now you can inspect its experiments / outcomes:
print("decision vars:", arch.experiments.columns.tolist())
print("outcome names:", list(arch.outcomes.keys()))

IsADirectoryError: [Errno 21] Is a directory: '/Users/tessahuysinga/Documents/1EPA Q4/Model-Based Decision Making/EPA141A/final assignment/archives'

In [38]:
import glob

archive_files = glob.glob("./archives/*.tar.gz")

archives = ArchiveLogger.load_archives(archive_files)



TypeError: expected str, bytes or os.PathLike object, not list